# Homeostasis à-la-SparseNet on the activation probability

In this notebook, we test the convergence of SparseNet as a function of different learning parameters. This shows the relative robusteness of this method according to the coding parameters, but also the importance of homeostasis to obtain an efficient set of filters.

See also :
* http://blog.invibe.net/posts/2015-05-05-reproducing-olshausens-classical-sparsenet.html for a description of how SparseNet is implemented in the scikit-learn package
* http://blog.invibe.net/posts/2015-05-06-reproducing-olshausens-classical-sparsenet-part-2.html for a descrtiption of how we managed to implement the homeostasis

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
from shl_scripts.shl_experiments import SHL_set
tag = 'HAP'
experiments = SHL_set({})

In [4]:
%%writefile {experiments.shl.data_cache}/{tag}.py

import numpy as np
np.set_printoptions(precision=2, suppress=True)

try:
    import sys
    command = sys.argv[1]
except:
    command = 'run'

from shl_scripts.shl_experiments import SHL_set
tag = 'HAP'
#opts = dict(homeo_method=tag, eta_homeo=0.01, alpha_homeo=0.7, verbose=0)
opts = dict(homeo_method=tag, verbose=0)

experiments = SHL_set(opts, tag=tag)

variables = ['eta', 'alpha_homeo']
variables = ['eta', 'alpha_homeo', 'eta_homeo', 'l0_sparseness']
variables = ['eta', 'alpha_homeo', 'eta_homeo']

if command == 'run':
    experiments.run(variables=variables)

if command == 'plot':
    import matplotlib.pyplot as plt

    #fig, ax = experiments.scan(variable='eta', list_figures=[], display='dynamic')
    #fig, ax = experiments.scan(variable='eta', list_figures=[], display='final')
    list_figures = []
    list_figures = ['show_dico']
    display_variables = ['error', 'logL', 'cputime']
    for variable in variables:
        experiments = SHL_set(opts, tag=tag)
        experiments.scan(variable=variable, list_figures=list_figures, display='')
        plt.show()

        for display_variable in display_variables:
            print('Time evolution of ', display_variable)
            fig, ax = experiments.scan(variable=variable, list_figures=[], display='dynamic', display_variable=display_variable)
            plt.show()
        plt.show()

        for display_variable in display_variables:
            print('Comparison at the end of learning for ', display_variable)
            fig, ax = experiments.scan(variable=variable, list_figures=[], display='final', 
                                       label=tag, display_variable=display_variable)
            plt.show()
        

Overwriting data_cache/HAP.py


In [5]:
!ls -l {experiments.shl.data_cache}/{tag}*
!rm -fr {experiments.shl.data_cache}/{tag}*lock*

-rw-r--r--  1 laurentperrinet  staff  1702 Mar  8 15:50 data_cache/HAP.py


In [6]:
toto = ['eta'] * 8

In [7]:
toto.extend(['tata'] * 8)

In [8]:
toto

['eta',
 'eta',
 'eta',
 'eta',
 'eta',
 'eta',
 'eta',
 'eta',
 'tata',
 'tata',
 'tata',
 'tata',
 'tata',
 'tata',
 'tata',
 'tata']

In [ ]:
%run {experiments.shl.data_cache}/{tag}.py run

['eta', 'eta', 'eta', 'eta', 'eta', 'eta', 'eta', 'alpha_homeo', 'alpha_homeo', 'alpha_homeo', 'alpha_homeo', 'alpha_homeo', 'alpha_homeo', 'alpha_homeo', 'eta_homeo', 'eta_homeo', 'eta_homeo', 'eta_homeo', 'eta_homeo', 'eta_homeo', 'eta_homeo'] [0.   0.   0.   0.   0.   0.01 0.01 0.1  0.16 0.25 0.4  0.63 1.01 1.6
 0.01 0.01 0.01 0.02 0.03 0.05 0.08]
eta 0.0005
eta 0.0007937005259840997
eta 0.0012599210498948732
eta 0.002
eta 0.003174802103936399
eta 0.005039684199579492
eta 0.008
alpha_homeo 0.1
alpha_homeo 0.15874010519681994
alpha_homeo 0.25198420997897464
alpha_homeo 0.4
alpha_homeo 0.6349604207872798
alpha_homeo 1.0079368399158983
alpha_homeo 1.6
eta_homeo 0.005
eta_homeo 0.007937005259840996
eta_homeo 0.012599210498948733
eta_homeo 0.02
eta_homeo 0.031748021039363986
eta_homeo 0.050396841995794917
eta_homeo 0.08


In [ ]:
%run {experiments.shl.data_cache}/{tag}.py plot

## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts, pandas